In [ ]:
!pip install -qU elasticsearch sentence-transformers==2.7.0

In [ ]:
# get the Elasticsearch client
from elasticsearch import Elasticsearch
from getpass import getpass

ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

In [ ]:
import json
from urllib.request import urlopen
from sentence_transformers import SentenceTransformer

if NBTEST["notebook"] in [
    "01-keyword-querying-filtering.ipynb",
    "02-hybrid-search.ipynb",
    "06-synonyms-api.ipynb",
]:
    # these tests need book_index to exist ahead of time
    client.indices.delete(index="book_index", ignore_unavailable=True)

    mappings = {
        "properties": {
            "title_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": "true",
                "similarity": "cosine",
            }
        }
    }
    client.indices.create(index="book_index", mappings=mappings)

    url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json"
    response = urlopen(url)
    books = json.loads(response.read())

    model = SentenceTransformer("all-MiniLM-L6-v2")
    operations = []
    for book in books:
        operations.append({"index": {"_index": "book_index"}})
        # Transforming the title into an embedding using the model
        book["title_vector"] = model.encode(book["title"]).tolist()
        operations.append(book)
    client.bulk(index="book_index", operations=operations, refresh=True)